### Quick check for linkrot in the harvested endpoints, both sets

using a head request to start

With timeouts and capturing redirects


In [1]:
import glob
import json
import os
import requests
from datetime import datetime

import logging
reload(logging)

logger = logging.getLogger(__name__)
handler = logging.FileHandler(filename="harvest_linkrot_%s.log" % datetime.now().strftime('%Y%m%d-%H%M'), mode="a", encoding="UTF-8")
logger.addHandler(handler)
logger.setLevel(logging.INFO)

files = glob.glob('../solr_superset/docs/*.json')

for i, f in enumerate(files):
    if i % 5000 == 0 and i > 0:
        print '{0} complete'.format(i)
    
    with open(f, 'r') as g:
        data = json.loads(g.read())
    
    url = data['url']
    
    try:
        req = requests.head(url, allow_redirects=True, timeout=10)
    except Exception as ex:
        logger.error('ERROR :: {0} :: {1} :: '.format(url, ex))
        continue
        
    if req.url != url:
        logger.info('REDIRECT :: {0} :: {1} :: {2}'.format(url, req.status_code, req.url))
    
    logger.info('RESPONSE :: {0} :: {1} :: '.format(url, req.status_code))


//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issue

5000 complete
10000 complete
15000 complete
20000 complete

//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issue


25000 complete

//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning



30000 complete

//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning



35000 complete
40000 complete
45000 complete


//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connection.py:251: SecurityWarning: Certificate has no `subjectAltName`, falling back to check for a `commonName` for now. This feature is being removed by major browsers and deprecated by RFC 2818. (See https://github.com/shazow/urllib3/issues/497 for details.)
  SecurityWarning


### TODOS

now that the set is finished (multithreading, fine, yes, one off things, etc)

- ~~run the ssl cert errors with verify=False just to get a good response~~
- ~~add a better flag for redirects (remove duplicate entry, but still tag)~~
- add the harvest date (first harvest, last year, vs second harvest this year) (grep in the folder? or ping solr? url encoding).




In [35]:
import csv
import requests
import subprocess

def get_code(status):
    if status.startswith('4'):
        return 400
    elif status.startswith('3'):
        return 300
    elif status.startswith('2'):
        return 200
    elif status.startswith('5'):
        return 500
    return -9999
    

data = []

with open('harvest_linkrot_20150504-1940.csv', 'rb') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        status = row['status'].strip()
        url = row['url'].strip()
        rsp_type = row['response'].strip()
        redirect = row['redirect'].strip()
    
        data.append((url, status, rsp_type, redirect))

# run the ssl without verify
# add a new status code as int to collapse sets
# skip if redirect, but check for a redirect and collapse rows
new_data = []
for url, status, rsp_type, redirect in data:
    row = {
        'url': url,
        'status_code': get_code(status),
        'redirected_url': '',
        'error': status if rsp_type == 'ERROR' or get_code(status) < 0 else ''
    }
    
    if 'SSL' in status:
        try:
            req = requests.head(url, verify=False)
            row['status_code'] = get_code(req.status_code)
        except Exception as Ex:
            #print url
            #print 'EXCEPTION:', ex
            row['error'] = ex.message
    elif redirect:
        continue
    else:
        redirected = [a for a in data if a[0] == url and a[2] == 'REDIRECT']
        if redirected and redirected[0][3] != url:
            row['redirected_url'] = redirected[0][3]
    
    new_data.append(row)



//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:734: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:734: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:734: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
//anaconda/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:734: InsecureRequestWarning: Unverified H

In [36]:
[n for n in new_data if n['redirected_url']][:5]

[{'error': '',
  'redirected_url': 'https://wordpress.org/showcase/web-designer-wall/feed/',
  'status_code': 200,
  'url': 'http://wordpress.org/showcase/web-designer-wall/feed/'},
 {'error': '',
  'redirected_url': 'https://wordpress.org/themes/flat/feed/',
  'status_code': 200,
  'url': 'http://wordpress.org/themes/rss/topic/flat'},
 {'error': '',
  'redirected_url': 'https://bbpress.org/xmlrpc.php?rsd',
  'status_code': 200,
  'url': 'http://bbpress.org/xmlrpc.php?rsd'},
 {'error': '',
  'redirected_url': 'http://gcmd.gsfc.nasa.gov/KeywordSearch/Titles.do?KeywordPath=Parameters%7CAGRICULTURE%7CAGRICULTURAL%20ENGINEERING%7CFARM%20STRUCTURES&Portal=GCMD&clientId=null&Output=atom',
  'status_code': 200,
  'url': 'http://gcmd.gsfc.nasa.gov/KeywordSearch/Titles.do?KeywordPath=Parameters|AGRICULTURE|AGRICULTURAL ENGINEERING|FARM STRUCTURES&Portal=GCMD&clientId=null&Output=atom'},
 {'error': '',
  'redirected_url': 'https://reslife.wustl.edu/?option=com_search&view=article&id=753&Itemid=3

In [37]:
with open('harvest_linkrot_20150504-1940_processed.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames=['url', 'status_code', 'redirected_url', 'error'])
    writer.writeheader()
    for row in new_data:
        writer.writerow(row)

In [ ]:
import subprocess
import os
import json

# url = 'http://wordpress.org/themes/rss/topic/flat'
# s = subprocess.Popen('grep -rl ../solr_superset/docs -e "%s"' % url, shell=True, stdout=subprocess.PIPE)
# s.communicate()

# anyway, forgot to store the md5 so we're doing this the hard way (and slow way, yes ruth, thank you)
def grep_url(url):
    s = subprocess.Popen('grep -rl ../solr_superset/docs -e "%s"' % url, shell=True, stdout=subprocess.PIPE)
    o = s.communicate()
    
    return o[0]

def get_date(filepath, url):
    with open(filepath, 'r') as f:
        data = json.loads(f.read())

    source_url = data['url']
    if source_url != url:
        return ''
    tstamp = data.get('tstamp', '')
    tdate = data.get('date', '')
    
    return tstamp if tstamp else tdate
    
    
mapped_data = []
for row in new_data:
    url = row['url']
    
    finds = grep_url(url)
    if finds:
        # get the date/timestamp from the json where this url is the source_url
        finds = finds.split('\n')
        
        harvest_date = ''
        for find in finds:
            d = get_date(find, url)
            if d:
                harvest_date = d
                break
        
        if not harvest_date:
            print url
            print finds
            print
            print
        
            continue
            
        row['harvested'] = harvest_date
        mapped_data.append(row)
    else:
        print 'not found', url

mapped_data[5:10]

### on to the eda

which i gave up on in pandas because sql is faster for me. 

In [19]:
import os
import pandas as pd
import numpy as np

pd.set_option('precision',2)

df = pd.read_csv('harvest_linkrot_20150504-1940.log', delimiter = ' ::', 
                 names=['response', 'url', 'status', 'redirect'], na_values='-9999')

df.sort('url')

,response,url,status,redirect
28880,RESPONSE,http://107.20.228.18/arcgis/services/Wetlands...,200,NaN
36203,RESPONSE,http://10ncee.org/?format=feed&type=atom,200,NaN
5191,RESPONSE,http://10ncee.org/?format=feed&type=rss,200,NaN
28805,RESPONSE,http://174.129.234.208/ArcGIS/services/retail...,200,NaN
11781,RESPONSE,http://2006.weblogawards.org/atom.xml,200,NaN
45070,RESPONSE,http://2006.weblogawards.org/index.xml,200,NaN
40186,RESPONSE,http://2012.the-embo-meeting.org/index7b17.ht...,200,NaN
48101,REDIRECT,http://2014.columbus.wordcamp.org/comments/feed/,200,https://columbus.wordcamp.org/2014/comments/f...
48102,RESPONSE,http://2014.columbus.wordcamp.org/comments/feed/,200,NaN
38172,REDIRECT,http://2014.columbus.wordcamp.org/feed/,200,https://columbus.wordcamp.org/2014/feed/


In [8]:
len(df[df['response'] == 'ERROR'])

2954

In [20]:
len(df[(df['response'] == 'REDIRECT') & (df['redirect'] != 'NaN')])

1829

In [21]:
len(df[(df['response'] == 'REDIRECT') & (df['redirect'] == 'NaN')])

0

In [22]:
df.to_csv('harvest_linkrot_20150504-1940.csv')

In [24]:
r = requests.head('http://ladsweb.nascom.nasa.gov/opendap/subsets/ab_GC-NET_Peterman_ELA/2014/MYS29/catalog.xml', verify=False)

r.status_code

302

In [25]:
r.headers

{'via': '1.1 hut-wsa5.colorado.edu:80 (Cisco-IronPort-WSA/7.5.1-079)', 'keep-alive': 'timeout=5, max=100', 'server': 'Apache/2.2.29', 'connection': 'keep-alive', 'location': 'https://ladsweb.nascom.nasa.gov/opendap/subsets/ab_GC-NET_Peterman_ELA/2014/MYS29/catalog.xml', 'date': 'Tue, 05 May 2015 23:16:03 GMT', 'content-type': 'text/html; charset=iso-8859-1'}